In [1]:
import praw
import configparser
import datetime
import pandas as pd 
from tqdm import tqdm
import time

In [2]:
# pip install --upgrade praw

In [3]:
config = configparser.ConfigParser()

In [4]:
reddit = praw.Reddit("bot1", user_agent="your user name")

In [35]:
subreddit = reddit.subreddit('diabetes_t1') # choose your subreddit

In [36]:
# extract hot posts
hot_posts = subreddit.hot(limit = 1000)
result = []
comments = []

for submission in tqdm(hot_posts):
    time.sleep(1)
    if not submission.stickied: # remove pinned posts
        created_time = submission.created_utc
        try:
            result.append({'ID':submission.id,
                           'Author':submission.author_fullname,
                       'Created_at':datetime.datetime.fromtimestamp(created_time),
                       'Title': submission.title,
                       'self_text': submission.selftext,
                       'comments': comments
                     })
            submission.comments.replace_more(limit=0)
            for top_level_comment in submission.comments:
                comments.append(top_level_comment.body)
            comments = []
          
        except AttributeError:
            continue
                
df_hot = pd.DataFrame(result)
df_hot.to_csv('Reddit_hot_t1.csv')

993it [23:07,  1.40s/it]


In [5]:
# extract the latest posts or posts after a submission_id
def extract_posts(subreddit, limit, submission_id):
    result = []
    comments = []

    if submission_id is None:
        new_posts = subreddit.new(limit=limit)
    else:
        new_posts = subreddit.new(limit=limit, params={'before': submission_id})

    for submission in tqdm(new_posts):
        time.sleep(1)
        if not submission.stickied:
            created_time = submission.created_utc
            try:
                result.append({
                    'ID': submission.id,
                    'Author': submission.author_fullname,
                    'Created_at': datetime.datetime.fromtimestamp(created_time),
                    'Title': submission.title,
                    'self_text': submission.selftext,
                    'comments': comments
                })
                submission.comments.replace_more(limit=0)
                for top_level_comment in submission.comments:
                    comments.append(top_level_comment.body)
                comments = []

            except AttributeError:
                continue
 
    return result


result = extract_posts(subreddit, limit=1000)
df = pd.DataFrame(result)
df.to_csv('Reddit_new.csv')



In [ ]:
# After this you can loop back to get the latest posts after your last submission_id. Here is an example:
submission_id = 't3_'+ df.iloc[0]['ID'] # submission if formed as 't3_' + the post ID
new_df = extract_posts(subreddit, limit = None, submission_id=submission_id)
new_df = pd.DataFrame(new_df)
df = pd.concat([new_df,df], ignore_index=True) 
df.drop_duplicates(subset='ID', inplace=True)